# Real Time Tribe

The `RealTimeTribe` class provides (near) real-time matched-filter methods.  It subclasses EQcorrscan's `Tribe` class and therefore
provides all the same methods, as well as real-time methods. We simply make a `RealTimeTribe` from a `Tribe` and some streaming client.
RT-EQcorrscan provides a `_Streaming` abstract base class outlining the basic attributes and methods a streaming client needs to have for
real-time matched-filtering.  The `RealTimeClient` is a mix of obspy's `EasySeedLinkClient` and the `_Streaming` abc, providing
access to seedlink services for real-time matched-filtering.

In this example we will see what seismicity is going on associated with the Rigdecrest sequence in California.

In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from eqcorrscan import Tribe
from eqcorrscan.utils.catalog_utils import filter_picks
from rt_eqcorrscan import RealTimeTribe
from rt_eqcorrscan.streaming import RealTimeClient

eq_client = Client("NCEDC")  # IRIS doesn't provide picks
client = Client("IRIS")  # We want to use IRIS for waveform data
rt_client = RealTimeClient(
    server_url="rtserve.iris.washington.edu",
    buffer_capacity=1200.) # NCEDC doesn't provide a public seedlink

template_catalog = eq_client.get_events(
    latitude=35.7, longitude=-117.4, maxradius=0.2, 
    starttime=UTCDateTime(2019, 7, 1), endtime=UTCDateTime(2019, 7, 15), 
    minmagnitude=4.5, includearrivals=True)
_ = template_catalog.plot(projection="local")

<Figure size 640x480 with 2 Axes>

We first have to generate a `Tribe` from this catalog. This will take a little while, and is one of the key motivations for using a `TemplateBank` of
previously recorded earthquakes.

For speed we will filter the catalog

In [2]:
unsupported_networks = {'BK', 'CI', 'NC', 'WR'} # IRIS doesn't provide everything.
all_networks = {p.waveform_id.network_code for event in template_catalog for p in event.picks}
filtered_catalog = filter_picks(
    template_catalog, top_n_picks=5, 
    networks=list(all_networks.difference(unsupported_networks)),
    channels=["EHZ", "EHN", "EHE", "EH1", "EH2", "HHZ", "HHN", "HHE", "HH1", "HH2"])
tribe = Tribe().construct(
    method="from_client", catalog=filtered_catalog, client_id=client,
    lowcut=2., highcut=15., samp_rate=50., filt_order=4, prepick=0.5, length=4.,
    swin="all", process_len=300.)

/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.7/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
Pick for LVA2.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for PFO.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for RDM.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for BEN.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for BHP.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for LVA2.HHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
Pick for PFO.HHZ has no 

We also need an inventory of stations that will be used.

In [3]:
seed_ids = {tuple(pick.waveform_id.get_seed_string().split('.'))
            for event in filtered_catalog for pick in event.picks}
bulk = [
    (sid[0], sid[1], sid[2], sid[3], UTCDateTime(2019, 7, 1), 
     UTCDateTime(2019, 7, 10)) for sid in seed_ids]
inventory = client.get_stations_bulk(bulk, level="channel")

Finally we can make the real-time tribe (note that plotting isn't supported in the notebook, but if you run this outside of the notebook and turn `plot=True` then you will get a bokeh live updating plot:

In [4]:
rt_tribe = RealTimeTribe(
        tribe=tribe, inventory=inventory, rt_client=rt_client,
        detect_interval=20, plot=False, plot_options=dict(plot_length=600.))

And we can run the (near) real-time detection!

In [5]:
import logging

logging.basicConfig(
    level="INFO", format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s"
)

rt_tribe.run(
    threshold=0.25, threshold_type="av_chan_corr", trig_int=10., 
    max_run_length=1200)  # Max_run_length will stop the process - leave unset to run forever.

2019-07-19 06:47:50,869	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	sending: requesting INFO level CAPABILITIES
2019-07-19 06:47:51,048	rt_eqcorrscan.streaming.streaming	INFO	Started streaming
2019-07-19 06:47:51,049	rt_eqcorrscan.rt_match_filter	INFO	Started real-time streaming
2019-07-19 06:47:51,050	rt_eqcorrscan.rt_match_filter	INFO	Detection will use the following data: {'AZ.RDM..HHZ', 'NN.BEN..EHZ', 'AZ.PFO..HHZ', 'NN.BHP..EHZ', 'AZ.LVA2..HHZ'}
2019-07-19 06:47:51,050	rt_eqcorrscan.rt_match_filter	INFO	Sleeping for 605.00s while accumulating data
2019-07-19 06:47:51,409	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	requesting next available data
2019-07-19 06:47:51,947	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	requesting next available data
2019-07-19 06:47:52,488	 obspy.clients.seedlink [rtserve.iris.washington.edu:18000]	INFO	requesting next available data
2019-07-19 06:47:53,027	 obspy.clients.seedlink [rtserve.iri

Party of 1 Families.

This will store detections in a `detections` directory.